In [1]:
!pip install "pinecone[grpc]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


Setting up model API key
Setting up Pinecone DB API key

In [2]:
import google.generativeai as genai
import pinecone
import os
import pandas as pd
# Set your API keys
genai.configure(api_key="")


In [3]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

In [4]:
pc = Pinecone(api_key="")

Create a sample of dataset comprising of ids and text.

In [5]:
# sample dataset
data = [
    {"id":"vec1","text":"Latent Dirichlet Allocation and Topic Modelling. The objective of the article is to understand the intuition behind LDA, the use cases and implementation."},
    {"id":"vec2","text":"Logistic Regression with R and Python. The objective of this article is to explore and understand Logistic Regression."},
    {"id":"vec3","text":"GCP and Cloud SQL.Cloud SQL is a fully managed database service. Cloud SQL is Google’s relational database."},
    {"id":"vec4","text":"Text Analysis on the reviews data of Indian products in Amazon.The objective of the article is to explore and analyze the reviews dataset of Indian products on Amazon with different NLP methodologies such as NLTK and Spacy."},
    {"id":"vec5","text":"Google Cloud Spanner.Spanner is a NewSQL database. NewSQL databases combine the scalability and high availability of NoSQL with a relational model, transactional support, and SQL of RDBMS."},
    {"id":"vec6","text":"The journey of Database from On Premise to Cloud.Cloud is about how you do computing, not where you do computing by Paul Maritz, CEO of VMware"},
    {"id":"vec7","text":"The evolution of RDBMS to NoSQL in the Big Data era.In the last few decades, RDBMS database was ubiquitous. For decades the relational databases had been the database management model when required as part of building an application."},
    {"id":"vec8","text":"BigqueryML and Logistic Regression. In this article will explore the Logistic Regression with SQL in BigQueryML, using the Heart dataset from Kaggle."},
    {"id":"vec9","text":"BigQuery and DataLab. We can work directly with BigQuery from Jupyter notebook in DataLab."},
    {"id":"vec10","text":"Loading data with csv file in Google Bigquery.In Google Cloud BigQuery we can load data from the csv file. We can create a dataset under the Google Project. Within the dataset we create tables."}
]

Make a call to the text embedding model for fetching the vector embeddings for the text

In [6]:
text = "Hello world"
result = genai.embed_content(model="models/text-embedding-004", content=text)

# Print just a part of the embedding to keep the output manageable
print(str(result['embedding'])[:50], '... TRIMMED]')

[0.013168517, -0.00871193, -0.046782672, 0.0006996 ... TRIMMED]


When we invoke the embedding model and pass "Hello world" as input text, we can
see the vector embeddings gets generated

Next, we will pass the sample data set into the embedding model.
Only passing the text.

In [7]:
#embeddings
result = genai.embed_content(
    model="models/text-embedding-004",
    content=[d['text'] for d in data])

for embedding in result['embedding']:
  print(str(embedding)[:50], '... TRIMMED]')


[-0.006170012, -0.033896714, -0.03969266, -0.03727 ... TRIMMED]
[0.020224523, -0.042412862, -0.015189768, -0.04505 ... TRIMMED]
[-0.018772395, 0.04788018, -0.023642184, 0.0127538 ... TRIMMED]
[-0.008478723, -0.023181247, -0.008485958, -0.0150 ... TRIMMED]
[0.036288094, 0.035432965, -0.010770451, 0.0430433 ... TRIMMED]
[0.0102558, 0.024010697, -0.01229662, -0.014475837 ... TRIMMED]
[-0.013332998, -0.0031336746, -0.008527822, 0.0136 ... TRIMMED]
[0.028458199, -0.019792574, -0.017109863, -0.00104 ... TRIMMED]
[0.005328373, -0.035129994, -0.077786066, -0.01380 ... TRIMMED]
[0.007038744, -0.010587484, -0.054532643, -0.00167 ... TRIMMED]


Storing the embeddings in the pandas dataframe

In [8]:
df = pd.DataFrame(columns=['id','text','embeddings'])
print(df)

Empty DataFrame
Columns: [id, text, embeddings]
Index: []


In [9]:
for embedding in result['embedding']:
  print(str(embedding)[:50], '... TRIMMED]')
  print(embedding)

df['embeddings']=result['embedding']

[-0.006170012, -0.033896714, -0.03969266, -0.03727 ... TRIMMED]
[-0.006170012, -0.033896714, -0.03969266, -0.037279483, -0.009010463, 0.008979701, -0.017441928, -0.047970876, 0.03198561, -0.060264412, 0.061081048, 0.035346635, -0.007333795, -0.07334046, 0.055596218, -0.050589092, 0.06515891, 0.039128654, -0.064327896, 0.029650452, 0.010282783, -0.006113909, -0.01312367, -0.01258974, 0.0017063376, -0.015002138, -0.01223897, -0.060316913, -0.016674506, -0.0067457813, 0.0244749, 0.012781415, 0.036423583, -0.003164387, -0.019217627, 0.0044770506, -0.00089762855, 0.00025486096, 0.01981681, -0.05463308, -0.028232537, -0.07280645, -0.029550258, 0.045651287, -0.0638895, -0.00864239, 0.010262211, 0.006762447, -0.024273336, -0.0035741844, 0.025532223, -0.028898206, -0.065579206, 0.05354344, -0.09902266, 0.010742718, -0.0018287536, 0.019106546, 0.057249527, -0.029033544, -0.015867779, -0.02988177, 0.010870901, -0.016694212, 0.010505659, -0.0008132139, -0.03032879, -0.00047294094, -0.01873421, 0.0

In [10]:
type(data)

list

In [11]:
df1 = pd.DataFrame(data)
print(df1)

      id                                               text
0   vec1  Latent Dirichlet Allocation and Topic Modellin...
1   vec2  Logistic Regression with R and Python. The obj...
2   vec3  GCP and Cloud SQL.Cloud SQL is a fully managed...
3   vec4  Text Analysis on the reviews data of Indian pr...
4   vec5  Google Cloud Spanner.Spanner is a NewSQL datab...
5   vec6  The journey of Database from On Premise to Clo...
6   vec7  The evolution of RDBMS to NoSQL in the Big Dat...
7   vec8  BigqueryML and Logistic Regression. In this ar...
8   vec9  BigQuery and DataLab. We can work directly wit...
9  vec10  Loading data with csv file in Google Bigquery....


In [12]:
df1['embeddings']=result['embedding']
print(df1)

      id                                               text  \
0   vec1  Latent Dirichlet Allocation and Topic Modellin...   
1   vec2  Logistic Regression with R and Python. The obj...   
2   vec3  GCP and Cloud SQL.Cloud SQL is a fully managed...   
3   vec4  Text Analysis on the reviews data of Indian pr...   
4   vec5  Google Cloud Spanner.Spanner is a NewSQL datab...   
5   vec6  The journey of Database from On Premise to Clo...   
6   vec7  The evolution of RDBMS to NoSQL in the Big Dat...   
7   vec8  BigqueryML and Logistic Regression. In this ar...   
8   vec9  BigQuery and DataLab. We can work directly wit...   
9  vec10  Loading data with csv file in Google Bigquery....   

                                          embeddings  
0  [-0.006170012, -0.033896714, -0.03969266, -0.0...  
1  [0.020224523, -0.042412862, -0.015189768, -0.0...  
2  [-0.018772395, 0.04788018, -0.023642184, 0.012...  
3  [-0.008478723, -0.023181247, -0.008485958, -0....  
4  [0.036288094, 0.035432965, -

In [13]:
print(df)

    id text                                         embeddings
0  NaN  NaN  [-0.006170012, -0.033896714, -0.03969266, -0.0...
1  NaN  NaN  [0.020224523, -0.042412862, -0.015189768, -0.0...
2  NaN  NaN  [-0.018772395, 0.04788018, -0.023642184, 0.012...
3  NaN  NaN  [-0.008478723, -0.023181247, -0.008485958, -0....
4  NaN  NaN  [0.036288094, 0.035432965, -0.010770451, 0.043...
5  NaN  NaN  [0.0102558, 0.024010697, -0.01229662, -0.01447...
6  NaN  NaN  [-0.013332998, -0.0031336746, -0.008527822, 0....
7  NaN  NaN  [0.028458199, -0.019792574, -0.017109863, -0.0...
8  NaN  NaN  [0.005328373, -0.035129994, -0.077786066, -0.0...
9  NaN  NaN  [0.007038744, -0.010587484, -0.054532643, -0.0...


Create an index in pinecone DB with the name "demoidx".
the dimensions for the vector embeddings is 1024,
the distance similarity is cosine

In [14]:
#create an index
index_name = "demoidx"

if not pc.has_index(index_name):
  pc.create_index(
      name = index_name,
      dimension=1024,
      metric="cosine",
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

In [15]:
#wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
  time.sleep(1)

Once the index is ready, we will add the vector embeddings via upsert.
Also metadata will be stored comprising of the text

In [16]:
# to add vectors to the index, uosert operation can be used.
# the upsert operation either adds a new vector to the index or updates an existing vector if a vector with the same ID laready exists.
index = pc.Index(index_name)

In [17]:
# prepare the records for upsert
# Each contains an 'id', the vector 'values'
# and the original text and category as 'metadata'
records = []
for idx in range(len(df1)):
  print(f"col3: {df1.iloc[idx]['embeddings']}")
  print(f"col1: {df1.iloc[idx]['id']}, col2: {df1.iloc[idx]['text']}")
  print("Next Record")

col3: [-0.006170012, -0.033896714, -0.03969266, -0.037279483, -0.009010463, 0.008979701, -0.017441928, -0.047970876, 0.03198561, -0.060264412, 0.061081048, 0.035346635, -0.007333795, -0.07334046, 0.055596218, -0.050589092, 0.06515891, 0.039128654, -0.064327896, 0.029650452, 0.010282783, -0.006113909, -0.01312367, -0.01258974, 0.0017063376, -0.015002138, -0.01223897, -0.060316913, -0.016674506, -0.0067457813, 0.0244749, 0.012781415, 0.036423583, -0.003164387, -0.019217627, 0.0044770506, -0.00089762855, 0.00025486096, 0.01981681, -0.05463308, -0.028232537, -0.07280645, -0.029550258, 0.045651287, -0.0638895, -0.00864239, 0.010262211, 0.006762447, -0.024273336, -0.0035741844, 0.025532223, -0.028898206, -0.065579206, 0.05354344, -0.09902266, 0.010742718, -0.0018287536, 0.019106546, 0.057249527, -0.029033544, -0.015867779, -0.02988177, 0.010870901, -0.016694212, 0.010505659, -0.0008132139, -0.03032879, -0.00047294094, -0.01873421, 0.02415542, -0.0072802524, -0.0154384235, -0.046027616, 0.093

In [18]:
records = []
for idx in range(len(df1)):
  records.append({
      "id":df1.iloc[idx]['id'],
      "values":df1.iloc[idx]['embeddings'],
      "metadata":{
          "source_text":df1.iloc[idx]['text']
      }
  })

In [19]:
#upsert the records into the index
index.upsert(
    vectors=records
)

PineconeException: UNKNOWN:Error received from peer  {grpc_message:"Vector dimension 768 does not match the dimension of the index 1024", grpc_status:3, created_time:"2025-02-16T22:42:34.285292184+00:00"}

we see the error message ""Vector dimension 768 does not match the dimension of the index 1024". this means we have created the vector embeddings with the dimension of 768 but the pinecone db index was created with the dimensions of 1024.
Will drop the index and create a new index with dimensions of 768

In [20]:
#delete tge index
pc.delete_index("demoidx")

In [21]:
#create an index
index_name = "demoidx"

if not pc.has_index(index_name):
  pc.create_index(
      name = index_name,
      dimension=768,
      metric="cosine",
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

In [22]:
#upsert the records into the index
index.upsert(
    vectors=records
)

upserted_count: 10

Verifying the index stats
it shows the dimensions as 768

In [23]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [24]:
# pinecone is eventually consistent

In [25]:
# vector search

In [26]:
# Q How to query the index and get similar vectors in Pinecone?

In [27]:
# define the query
query = "Semantic layer and Business Intelligence"

In [28]:
# convert the query. into a numerical vector that pinecone can search with
#embeddings
query_embeddings = genai.embed_content(
    model="models/text-embedding-004",
    content=query)

In [29]:
# search the index for the top1 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=1,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec7',
              'metadata': {'source_text': 'The evolution of RDBMS to NoSQL in '
                                          'the Big Data era.In the last few '
                                          'decades, RDBMS database was '
                                          'ubiquitous. For decades the '
                                          'relational databases had been the '
                                          'database management model when '
                                          'required as part of building an '
                                          'application.'},
              'score': 0.45515248,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [30]:
# search the index for the top2 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=2,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec7',
              'metadata': {'source_text': 'The evolution of RDBMS to NoSQL in '
                                          'the Big Data era.In the last few '
                                          'decades, RDBMS database was '
                                          'ubiquitous. For decades the '
                                          'relational databases had been the '
                                          'database management model when '
                                          'required as part of building an '
                                          'application.'},
              'score': 0.45515248,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec9',
              'metadata': {'source_text': 'BigQuery and DataLab. We can work '
                                          'directly with BigQuery from Jupyter '
                                          'notebook in DataLab.

In [31]:
# search the index for the top3 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec7',
              'metadata': {'source_text': 'The evolution of RDBMS to NoSQL in '
                                          'the Big Data era.In the last few '
                                          'decades, RDBMS database was '
                                          'ubiquitous. For decades the '
                                          'relational databases had been the '
                                          'database management model when '
                                          'required as part of building an '
                                          'application.'},
              'score': 0.45515248,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec9',
              'metadata': {'source_text': 'BigQuery and DataLab. We can work '
                                          'directly with BigQuery from Jupyter '
                                          'notebook in DataLab.

In [32]:
# add another user query
query = "AI and Machine Learning"


In [33]:
# convert the query. into a numerical vector that pinecone can search with
#embeddings
query_embeddings = genai.embed_content(
    model="models/text-embedding-004",
    content=query)

In [34]:
# search the index for the top1 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=1,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec2',
              'metadata': {'source_text': 'Logistic Regression with R and '
                                          'Python. The objective of this '
                                          'article is to explore and '
                                          'understand Logistic Regression.'},
              'score': 0.40734386,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [35]:
# search the index for the top2 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=2,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec2',
              'metadata': {'source_text': 'Logistic Regression with R and '
                                          'Python. The objective of this '
                                          'article is to explore and '
                                          'understand Logistic Regression.'},
              'score': 0.40734386,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec1',
              'metadata': {'source_text': 'Latent Dirichlet Allocation and '
                                          'Topic Modelling. The objective of '
                                          'the article is to understand the '
                                          'intuition behind LDA, the use cases '
                                          'and implementation.'},
              'score': 0.3917798,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace

In [36]:
# search the index for the top3 vector
results = index.query(
    vector = query_embeddings['embedding'],
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec2',
              'metadata': {'source_text': 'Logistic Regression with R and '
                                          'Python. The objective of this '
                                          'article is to explore and '
                                          'understand Logistic Regression.'},
              'score': 0.40734386,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec1',
              'metadata': {'source_text': 'Latent Dirichlet Allocation and '
                                          'Topic Modelling. The objective of '
                                          'the article is to understand the '
                                          'intuition behind LDA, the use cases '
                                          'and implementation.'},
              'score': 0.3917798,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
            

In [ ]:
pc.delete_index(index_name)